Part 1: Training your Algorithm

We are going to be using our trained algorithm from Assigment #1. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Read .csv file 
airline = pd.read_csv('Invistico_Airline.csv')

#Dropped 
airline.drop('Class', axis=1, inplace=True)
airline.drop('Flight Distance', axis=1, inplace=True)
airline.drop('Departure/Arrival time convenient', axis=1, inplace=True)
airline.drop('Gate location', axis=1, inplace=True)
airline.drop('Online support', axis=1, inplace=True)
airline.drop('Ease of Online booking', axis=1, inplace=True)
airline.drop('Arrival Delay in Minutes', axis=1, inplace=True)

#Replaced
airline.replace('Male', 0, inplace=True)
airline.replace('Female', 1, inplace=True)
airline.replace('Personal Travel', 0, inplace=True)
airline.replace('Business travel', 1, inplace=True)
airline.replace('Loyal Customer', 0, inplace=True)
airline.replace('disloyal Customer', 1, inplace=True)
airline.replace('satisfied', 0, inplace=True)
airline.replace('dissatisfied', 1, inplace=True)

#Dropped NaN rows 
airline.dropna(axis=0, inplace=True)

In [ ]:
# We need to split our data into x and y. We use X to predict y.
X, y = airline.drop('satisfaction', axis=1), airline['satisfaction'].values

# Split the data into a training set and a testing set 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=87)

# Train our algorithm 
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(max_iter=200000)
logmodel.fit(X_train, y_train)

Part 2: Restful Web Service Setup

In [ ]:
# Start with imports 
from flask import Flask, request
app = Flask(__name__)

# The name of our URL will also be called <airline>. 

In [ ]:
# We are going to start with the first record of our dataframe without the satisfaction column 
airline.drop('satisfaction', axis=1).iloc[0]

In [ ]:
# We need to format our pandas series into a JSON string
airline.drop('satisfaction', axis=1).iloc[0].to_json(orient="index")

Part 3: Sample 

In [ ]:
# This will return a JSON record showing a sample rquest. 
# This JSON record will be copied and pasted in Postman to test Part 5. 
@app.get("/airline/sample")
def get_sample():
    return airline.drop('satisfaction', axis=1).iloc[0].to_json(orient="index")

Part 4: Explain

In [ ]:
# This will return a string that explains what and how your sample from part 3 is and how it should be modified. 
explanation = "The following columns were altered so that the algorithm can process the data: Gender, Type of Travel, Customer Type, and Satisfied.\n\nGender: 'Male' = 0 and 'Female' = 1.\nType of Travel: 'Personal Travel' = 0 and 'Business travel' = 1.\nCustomer Type: 'Loyal Customer' = 0 and 'disloyal Customer' = 1. " + "\n\nThe algorithm will return a 0 if the customer was satisifed with their flight or 1, if customer was dissatisfied with their flight." + "\nOtherwise, the remaining columns are to be rated from 0 to 5. 0 being the lowest and 5 being the highest."

@app.get("/airline/explain")
def get_explanation():
    return explanation

Part 5: Evaluate

In [ ]:
# This will return a string that explains what and how your sample from part 3 is and how it should be modified. 
@app.post("/airline/evaluate")
def test_sample():
    if request.is_json:
        person = request.get_json()
        dataframe = pd.DataFrame([person])
        result = logmodel.predict(dataframe)
        return '{}'.format(result[0])
    return {"error": "Request must be JSON"}

In [ ]:
if __name__ == '__main__':
    app.run()